# Predict new locations for UBS wealth management branch offices.

Union Bank of Switzerland (UBS) is looking to apply Machine Learning in predicting most likely new locations/zipcodes be opened with UBS wealth management branches in the USA

### keywords: data wrangling, PCA, under-sampling, cross-validation, cross-prediction, LogisticRegression, Naive Bayes, ensemble learning 

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import random
import copy
from sklearn.model_selection import KFold, cross_val_score,cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
from collections import Counter

C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


# Data wrangling

In [2]:
#Import original dataset
original_dataset=pd.read_csv('UBS_original_data.csv')
dataset=copy.deepcopy(original_dataset)
dataset.head()

,ZipCode,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,AsianPopulation,HawaiianPopulation,IndianPopulation,OtherPopulation,...,DeliveryTotal,PopulationEstimate,LandArea,WaterArea,BoxCount,SFDU,MFDU,CityDeliveryIndicator,MedicareCBSAType,MarketRatingAreaID
0,501,0,0,0,0,0,0,0,0,0,...,1,0,0.000,0.000,0,0,0,N,Metro,8
1,501,0,0,0,0,0,0,0,0,0,...,1,0,0.000,0.000,0,0,0,N,Metro,8
2,544,0,0,0,0,0,0,0,0,0,...,0,0,0.000,0.000,0,0,0,N,Metro,8
3,544,0,0,0,0,0,0,0,0,0,...,0,0,0.000,0.000,0,0,0,N,Metro,8
4,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1


In [3]:
#Remove the rows where the population are 0
dataset = dataset[dataset.Population != 0]
dataset.head()

,ZipCode,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,AsianPopulation,HawaiianPopulation,IndianPopulation,OtherPopulation,...,DeliveryTotal,PopulationEstimate,LandArea,WaterArea,BoxCount,SFDU,MFDU,CityDeliveryIndicator,MedicareCBSAType,MarketRatingAreaID
4,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1
5,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1
6,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1
7,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1
8,602,41520,15002,36828,2860,41265,42,32,291,2634,...,11165,24000,30.613,1.717,1502,5420,821,Y,Metro,1


In [4]:
#Reset the row index for later use
dataset.index=range(len(dataset))
#Replace the empty cell with the NaN
for j in range(dataset.columns.size):
    dataset.iloc[:,j].replace(r'\s+', np.nan, regex=True, inplace=True)

In [5]:
#Checking for missing values(NaN) in each feature
feature_nan_value=dataset.isnull().sum()
print(feature_nan_value)

ZipCode                            0
Population                         0
HouseholdsPerZipCode               0
WhitePopulation                    0
BlackPopulation                    0
HispanicPopulation                 0
AsianPopulation                    0
HawaiianPopulation                 0
IndianPopulation                   0
OtherPopulation                    0
MalePopulation                     0
FemalePopulation                   0
PersonsPerHousehold                0
AverageHouseValue                  0
IncomePerHousehold                 0
MedianAge                          0
MedianAgeMale                      0
MedianAgeFemale                    0
CityType                           0
NumberOfBusinesses                 0
NumberOfEmployees                  0
BusinessFirstQuarterPayroll        0
BusinessAnnualPayroll              0
BusinessEmploymentFlag         62853
GrowthRank                         0
GrowingCountiesA                   0
GrowingCountiesB                   0
G

In [6]:
#Drop those features with over 50% of the values being NaN 
dataset.drop(['BusinessEmploymentFlag'],axis=1,inplace=True)

In [7]:
#Fill the nan with its next value for the feature "MedicareCBSAType" 
dataset.MedicareCBSAType.fillna(method='bfill',inplace=True)

In [8]:
#Encode categorical features with dummy variables
dataset=pd.get_dummies(dataset)  

In [9]:
#Group by zip code, and select max value of each group
dataset=dataset.groupby('ZipCode',as_index=False).max()

In [10]:
#Input label/target data
#the zipcodes already opened with branches are labeled with "1", otherwise "0"
dataset_label=pd.read_csv('UBS_data_label.csv')
dataset_label.head()

,ZipCode,UBS_Open_Branch
0,1144,1
1,1608,1
2,1945,1
3,1960,1
4,2109,1


In [11]:
#Final clean data with the label
#Final_dataset=pd.merge(dataset,dataset_label.drop(['UBS_Branches'],axis=1), on='ZipCode',how='left')
Final_dataset=pd.merge(dataset,dataset_label, on='ZipCode',how='left')
Final_dataset.UBS_Open_Branch.fillna(0,inplace=True)
Final_dataset.to_csv('UBS_clean_data.csv',index=False)

In [12]:
# Import clean dataset
original_dataset = pd.read_csv('UBS_clean_data.csv')
original_dataset.head()

,ZipCode,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,AsianPopulation,HawaiianPopulation,IndianPopulation,OtherPopulation,...,CityType_C,CityType_N,CityType_P,CityType_U,CityType_Z,CityDeliveryIndicator_N,CityDeliveryIndicator_Y,MedicareCBSAType_Metro,MedicareCBSAType_Micro,UBS_Open_Branch
0,601,18570,6525,17479,663,18486,7,10,113,558,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,602,41520,15002,36828,2860,41265,42,32,291,2634,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
2,603,54689,21161,46501,5042,53877,135,35,313,4177,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,606,6615,2404,5979,371,6575,3,9,35,323,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,610,29016,10836,24510,2654,28789,57,31,200,2494,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


## Principal Component Analysis (PCA)

In [13]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler 

In [14]:
# Scale the features and set the values to a new variable
scaler = StandardScaler() 
original_dataset_features = scaler.fit_transform(original_dataset.drop(['ZipCode','UBS_Open_Branch'],axis=1))

C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [15]:
# Import PCA class
from sklearn.decomposition import PCA

In [16]:
#Find the suitable number of PCA components 
pca = PCA(random_state=7)
pca.fit(original_dataset_features)
#Get explained variance ratios from PCA using all features
exp_variance = pca.fit(original_dataset_features).explained_variance_ratio_

In [17]:
# Calculate the cumulative explained variance
cum_exp_variance = np.cumsum(exp_variance)

In [18]:
#Determine the number of components by 90% 
#of the cumulative explained variance  
n_components = len(cum_exp_variance[cum_exp_variance<0.9])
print(n_components)

19


In [19]:
#Perform PCA with the chosen number of components and project data onto components
pca = PCA(n_components)#, random_state=10)
pca.fit(original_dataset_features)
PCA_dataset_features= pca.transform(original_dataset_features)
PCA_dataset_features=pd.DataFrame(PCA_dataset_features)

In [20]:
#Add label and zip code index
PCA_dataset=PCA_dataset_features
PCA_dataset['UBS_Open_Branch']=original_dataset['UBS_Open_Branch']
PCA_dataset['ZipCode']=original_dataset['ZipCode']
#Add prediction coloumn
PCA_dataset['y_pred_prob']=0.0

In [21]:
#Number of records in class 0 and class 1
print('The size of class 1: {}'.format(len(PCA_dataset[PCA_dataset.UBS_Open_Branch == 0])))
print('The size of class 0: {}'.format(len(PCA_dataset[PCA_dataset.UBS_Open_Branch != 0]))) 

The size of class 1: 32695
The size of class 0: 264


## Binary classification problem on a highly imbalanced dataset

In [22]:
# Minority dataset
minority_dataset=PCA_dataset[PCA_dataset.UBS_Open_Branch != 0]
# Majority dataset
majority_dataset = PCA_dataset[PCA_dataset.UBS_Open_Branch == 0]

## Undersampling & cross-prediction 

For this specific problem, the information of zipcodes in majority class are requied for model training, on the other hand, any zipcode in majority class is possbile to be opened with a new branch.
Therefore, regular model training and prediction process lead to "data leakage", which refers to a mistake made by the creator of a machine learning model, in which they accidentally share information between the test and training data-sets, memorizes the training set data, and is easily able to correctly output the labels/values for those test data-set examples.
In order to avoid "data leakage", we employ an undersampling & cross-prediction scheme.

Divide the majority dataset into m=122 subsets, each subset size is n=268, we randomly select one subset of majority dataset, and combine it with the minority set as a train set to train the model with cross-validation, the cross-validation score is recored as the weight which is used in the final stage. 
Keep the remaining part of the majority dataset as the test set, predict the test set with the trained model.
<img src="cross_prediction2-1.png">
Repeat above process until all the m=122 subsets of majority dataset have been employed for model training. 
Finally calculate the weighted average of the predictions for each zipcode, 
those zipcodes with the top predictions are the ones to be opend wth the new branches most likely.

In [23]:

Monte_Carlo=2

monte_All_ZipCode_prediction=PCA_dataset.loc[:,['ZipCode','y_pred_prob']]
monte_All_ZipCode_prediction.sort_values('ZipCode',inplace=True,ascending=False)

for r in range(Monte_Carlo):
    print(r)
    
    #Shuffle the indeces of Majority dataset
    majority_index=list(majority_dataset.index)
    #random.seed(2337)
    random.shuffle(majority_index)

    #Parameters initialization

    simulation_rounds=122

    #Size of each subset in majority set
    subsetsize=268

    #Indeces of elements in each subset of majority set
    majority_subset_index=[]

    #Cross-validation score initialization
    Model_performance=pd.DataFrame(np.zeros((122,1)),columns=['Accuracy'])

    #Obtain indeces of elements in each subset of majority set
    for i in range(122):
        majority_subset_index.append(majority_index[i*subsetsize:(i+1)*subsetsize])

    #Initialize the final weighted prediction of each zipcode
    All_ZipCode_prediction=PCA_dataset.loc[:,['ZipCode','y_pred_prob']]

    #Initialize the prediction of each zipcode in a single round cross-prediction
    temp_single_round_prediction=PCA_dataset.loc[:,['ZipCode','y_pred_prob']] 

    #Train model with K-Fold cross validation
    kf = KFold(n_splits=10)#, random_state=10)

    #Classification model ensembling

    #Logistic regression
    logreg= LogisticRegression(solver='liblinear')#random_state=10
    #Naive bayes
    NB = GaussianNB()
    
#Cross prediction

    for j in range(simulation_rounds):

        #A subset of majoriry data set is combined with the minority set to form the train set
        majority_datasubset=majority_dataset.loc[majority_subset_index[j],:]
        dataset_train=majority_datasubset.append(minority_dataset,ignore_index=False)    
        X_train= dataset_train.iloc[:,1:-3].values ##features
        y_train = dataset_train.UBS_Open_Branch.values##labels


        #Train models using KFold cv
        #logit_score =cross_val_score(logreg, X_train, y_train, cv=kf,scoring='accuracy')
        #NB_score = cross_val_score(NB, X_train, y_train, cv=kf,scoring='accuracy')
        logit_score = cross_validate(logreg, X_train, y_train,  scoring='accuracy', cv=kf)
        NB_score=cross_validate(NB, X_train, y_train,  scoring='accuracy', cv=kf)
        ave_cv_logit_train_score=np.mean(logit_score['train_score'])
        ave_cv_logit_test_score=np.mean(logit_score['test_score'])
        ave_cv_NB_train_score=np.mean(NB_score['train_score'])
        ave_cv_NB_test_score=np.mean(NB_score['test_score'])
        
        #Select the average test score of the model with less likely overfitting
        #as cross validation score
        if (ave_cv_logit_train_score-ave_cv_logit_test_score)/ave_cv_logit_test_score>(ave_cv_NB_train_score-ave_cv_NB_test_score)/ave_cv_NB_test_score:
            cross_validation_score = np.mean(NB_score['test_score'])
        else:
            cross_validation_score = np.mean(logit_score['test_score'])
            
        #Predict the test set
        #The remaining part of the majority dataset which is not used in training process
        #is the test set   
        X_test=majority_dataset.drop(majority_subset_index[j],axis=0).iloc[:,1:-3].values
        X_test_ZipCode=majority_dataset.drop(majority_subset_index[j],axis=0).loc[:,['ZipCode','y_pred_prob']]

        #Select the model of less likely overfitting 
        #to predict the test set
        if (ave_cv_logit_train_score-ave_cv_logit_test_score)/ave_cv_logit_test_score>(ave_cv_NB_train_score-ave_cv_NB_test_score)/ave_cv_NB_test_score:
            NB.fit(X_train, y_train)           
            y_probas= NB.predict_proba(X_test)
        else:
            logreg.fit(X_train, y_train) 
            y_probas= logreg.predict_proba(X_test)

        X_test_ZipCode['y_pred_prob']=y_probas[:,1]

        #Merge the predictions on Zipcode
        single_round_predictions=pd.merge(temp_single_round_prediction,X_test_ZipCode, on='ZipCode',how='left')

        #Replace nan with 0
        single_round_predictions.iloc[:,-1].fillna(0,inplace=True)


        #Weighted averaging the prediction probability with the 
        #Cross validation score from each model
        All_ZipCode_prediction['y_pred_prob']+=single_round_predictions.iloc[:,-1]*cross_validation_score 
        Model_performance.iloc[j,:]=cross_validation_score
        ##print('simulation round {}: cross validation score is {}'.format(j,cross_validation_score)) 
        ##print('simu_round {}: \n c-v train_score is {} \n c-v test_score is {}'.format(j,np.around(scores['train_score'],decimals=3),np.around(scores['test_score'],decimals=3))) 
        print('Cross-validation for {}-th subset using logit: \n logit c-v train_score mean is {} \n logit c-v test_score mean is {}'.format(j,np.mean(logit_score['train_score']),np.mean(logit_score['test_score'])))
        print('Cross-validation for {}-th subset using NB: \n NB c-v train_score mean is {} \n NB c-v test_score mean is {}'.format(j,np.mean(NB_score['train_score']),np.mean(NB_score['test_score'])))
    All_ZipCode_prediction.sort_values('ZipCode',inplace=True,ascending=False)
    All_ZipCode_prediction['y_pred_prob']=All_ZipCode_prediction['y_pred_prob']/Model_performance.iloc[:,0].sum()
    
    monte_All_ZipCode_prediction['y_pred_prob']+= All_ZipCode_prediction['y_pred_prob']
    # print(All_ZipCode_prediction.y_pred_prob.head(10))
    # print(monte_All_ZipCode_prediction.y_pred_prob.head(10))

0
Cross-validation for 0-th subset using logit: 
 logit c-v train_score mean is 0.8796979411430719 
 logit c-v test_score mean is 0.7929769392033543
Cross-validation for 0-th subset using NB: 
 NB c-v train_score mean is 0.7391348782767446 
 NB c-v test_score mean is 0.706219426974144
Cross-validation for 1-th subset using logit: 
 logit c-v train_score mean is 0.8974506686699103 
 logit c-v test_score mean is 0.8494060097833683
Cross-validation for 1-th subset using NB: 
 NB c-v train_score mean is 0.7232658694455848 
 NB c-v test_score mean is 0.6836128581411601


C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125:

Cross-validation for 2-th subset using logit: 
 logit c-v train_score mean is 0.8840868790454313 
 logit c-v test_score mean is 0.8119147449336129
Cross-validation for 2-th subset using NB: 
 NB c-v train_score mean is 0.7393423362828767 
 NB c-v test_score mean is 0.7081062194269742


C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125:

Cross-validation for 3-th subset using logit: 
 logit c-v train_score mean is 0.895364733012465 
 logit c-v test_score mean is 0.845737246680643
Cross-validation for 3-th subset using NB: 
 NB c-v train_score mean is 0.730993789362427 
 NB c-v test_score mean is 0.6647449336128581
Cross-validation for 4-th subset using logit: 
 logit c-v train_score mean is 0.8757304705584332 
 logit c-v test_score mean is 0.7931167016072675
Cross-validation for 4-th subset using NB: 
 NB c-v train_score mean is 0.6961076510512662 
 NB c-v test_score mean is 0.6476589797344514


...

C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125:

Cross-validation for 115-th subset using logit: 
 logit c-v train_score mean is 0.8982848682314095 
 logit c-v test_score mean is 0.8323549965059399
Cross-validation for 115-th subset using NB: 
 NB c-v train_score mean is 0.7445602327023699 
 NB c-v test_score mean is 0.7060097833682739
Cross-validation for 116-th subset using logit: 
 logit c-v train_score mean is 0.8732213205684785 
 logit c-v test_score mean is 0.8118099231306779
Cross-validation for 116-th subset using NB: 
 NB c-v train_score mean is 0.7123941964168727 
 NB c-v test_score mean is 0.6719776380153738
Cross-validation for 117-th subset using logit: 
 logit c-v train_score mean is 0.8730169198382265 
 logit c-v test_score mean is 0.8043326345213139
Cross-validation for 117-th subset using NB: 
 NB c-v train_score mean is 0.692349385487548 
 NB c-v test_score mean is 0.6458071278825996


C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125:

Cross-validation for 118-th subset using logit: 
 logit c-v train_score mean is 0.8893056489723186 
 logit c-v test_score mean is 0.834346610761705
Cross-validation for 118-th subset using NB: 
 NB c-v train_score mean is 0.7405918886103371 
 NB c-v test_score mean is 0.7079315164220825
Cross-validation for 119-th subset using logit: 
 logit c-v train_score mean is 0.907683371039736 
 logit c-v test_score mean is 0.8438155136268344
Cross-validation for 119-th subset using NB: 
 NB c-v train_score mean is 0.7178287226701373 
 NB c-v test_score mean is 0.6646750524109015
Cross-validation for 120-th subset using logit: 
 logit c-v train_score mean is 0.893480577563089 
 logit c-v test_score mean is 0.8362683438155136
Cross-validation for 120-th subset using NB: 
 NB c-v train_score mean is 0.726602667691582 
 NB c-v test_score mean is 0.7061844863731657
Cross-validation for 121-th subset using logit: 
 logit c-v train_score mean is 0.898096541319088 
 logit c-v test_score mean is 0.836093

C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [24]:
All_ZipCode_prediction['y_pred_prob'] =monte_All_ZipCode_prediction['y_pred_prob']/ Monte_Carlo  

All_ZipCode_prediction.sort_values('y_pred_prob',inplace=True,ascending=False)

print(All_ZipCode_prediction.head(100).to_string(index=False))

ZipCode  y_pred_prob
  10036     0.992966
  60601     0.992626
  10010     0.992467
  80111     0.992412
  94103     0.992411
  10011     0.992403
  60654     0.992371
  10003     0.992351
  94111     0.992310
  43215     0.992302
  22102     0.992292
  60611     0.992227
  30309     0.992222
  20036     0.992214
  10021     0.992209
  20004     0.992194
  28202     0.992192
  98109     0.992180
   2210     0.992154
  90048     0.992139
  90245     0.992129
  70809     0.992127
  60603     0.992125
  94301     0.992117
  30303     0.992115
   2110     0.992088
  90210     0.992082
  33166     0.991864
  37027     0.991809
  77024     0.991800
  10004     0.991788
  78759     0.991770
  92121     0.991747
  98052     0.991734
  46204     0.991694
  75039     0.991666
  98101     0.991660
  10001     0.991620
  75093     0.991595
  15222     0.991585
  10016     0.991565
  89109     0.991561
  95054     0.991551
  77030     0.991519
  10014     0.991507
  30339     0.991477
  92108     0